In [0]:
dbutils.widgets.text("layer", "silver")
layer = dbutils.widgets.get("layer")

print("Running layer:", layer)

In [0]:
from pyspark.sql import functions as F
def create_user_features(events_df):
    user_features_df = events_df.groupBy("user_id").agg(
    F.count("*").alias("total_events"),
    F.count(F.when(F.col("event_type") == "purchase", 1)).alias("purchases"),
    F.sum("price").alias("total_spent"),
    F.avg("price").alias("avg_price")
    )
    return user_features_df

In [0]:
# Load Bronze Layer (Delta Table)
events = spark.table("workspace.ecommerce.events_delta")

print("Bronze table loaded successfully")

In [0]:
if layer == "silver":
    
    print("Creating Silver Layer Feature Table...")

    # Load Bronze
    events = spark.table("workspace.ecommerce.events_delta")

    # Create features
    features_df = create_user_features(events)

    # Write Silver
    features_df.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable("workspace.ecommerce.user_features_silver")

    print("Silver feature table created successfully!")

else:
    print("No matching layer selected.")